## Contrôle des feux de signalisation

Ce programme présente une implémentation d'un controller step-based décrit dans l'article "Guidelines for Action Space Definition in Reinforcement Learning-Based Traffic Signal Control Systems" de Maxime Treca, Julian Garbiso, Dominique Barth.

l'objet d'étude est une intersection composée de 4 routes qui arrivent de chaque direction Nord, Sud, Est, Ouest


<img src="files/figures/intersection.jpg" width="500px"/>

Un scénario dure 10 000 steps

In [37]:
import numpy as np

class IntersectionEnvironment:
    
    def __init__(self, d_min, d_max, n_step, controller_type = 'step-based'):
        #durée min et max d'une phase
        self.d_min = d_min
        self.d_max = d_max
        
        self.n_step = n_step
        
        #cycle Ψ
        self.signal_cycle = ['ψ1', 'ψT', 'ψ2', 'ψT'] 
        self.controller_type = controller_type
        
        #Durée de chaque phase de ψ
        self.phase_duration = {
            'ψ1': np.random.randint(d_min, d_max+1),
            'ψ2': np.random.randint(d_min, d_max+1),
            'ψT': 5
        }
        
        #Nombre de véhicules sur les 4 routes Nord, Sud, Est, Ouest
        self.approaches ={
            'N' :0,
            'S' :0,
            'W' :0,
            'E':0
        }
        
        #Indice et durée de la phase initiale
        self.current_phase_idx = 0
        self.current_phase_duration = self.phase_duration[self.signal_cycle[self.current_phase_idx]]
        self.time_since_last_phase = 0
        
        #Congestion associée à chaque phase
        self.congestion = {'ψ1': 0, 'ψ2': 0}
        self.old_congestion = {'ψ1': 0, 'ψ2': 0}
        self.cumulated_waiting_time = 0
        self.time_elapsed = 0
        
    def reset(self):
        self.phase_duration = {
            'ψ1': np.random.randint(self.d_min, self.d_max+1),
            'ψ2': np.random.randint(self.d_min, self.d_max+1),
            'ψT': 5
        }
        self.approaches ={
            'N' : 0,
            'S' : 0,
            'W' : 0,
            'E': 0
        }
        self.current_phase_idx = 0
        self.current_phase_duration = self.phase_duration[self.signal_cycle[self.current_phase_idx]]
        self.time_since_last_phase = 0
        self.congestion = {'ψ1': 0, 'ψ2': 0}
        self.old_congestion = {'ψ1': 0, 'ψ2': 0}
        self.cumulated_waiting_time = 0
        self.time_elapsed = 0
        return ('ψ1', self.phase_duration['ψ1'], self.congestion['ψ1'], self.congestion['ψ2'])
    
    def step(self, action):
        print( " t =  :", self.time_elapsed," phase", self.signal_cycle[self.current_phase_idx] )
        terminal = False
        reward = 0
        self.time_elapsed+=1
        self.time_since_last_phase += 1
        
        
        # Generate traffic demand
        self.generate_trafic(0.2,0.3)
        
        # Updating traffic 
        self.update_trafic()
        
        #Transition phase
        if self.time_since_last_phase == self.current_phase_duration and self.signal_cycle[self.current_phase_idx] == 'ψT' :
            self.time_since_last_phase = 0
            self.switch_phase()
         
        #Decision phase
        elif self.time_since_last_phase == self.current_phase_duration and self.signal_cycle[self.current_phase_idx] != 'ψT' :
            self.old_congestion = self.congestion #congestion at the last decision point
            if action == 0: # Extend the current phase
                if self.current_phase_duration == self.d_max:
                    self.time_since_last_phase = 0
                    self.switch_phase()
                elif self.current_phase_duration + self.n_step <= self.d_max:
                    print( "phase extended by ", self.n_step)
                    self.current_phase_duration += self.n_step
                elif self.current_phase_duration + self.n_step > self.d_max:
                    self.time_since_last_phase = 0
                    self.switch_phase()
                    
            else: # Switch to the next phase
                self.time_since_last_phase = 0
                self.switch_phase()

            
        self.congestion = self.update_congestion() #congestion at time t
        
        if self.signal_cycle[self.current_phase_idx] != 'ψT' :#reward is not calculated for phase ψT
            #difference between the congestion at time t and the congestion at the last decision point
            reward = self.old_congestion[self.signal_cycle[self.current_phase_idx]] - self.congestion[self.signal_cycle[self.current_phase_idx]]
        
        
        state = (self.signal_cycle[self.current_phase_idx],self.current_phase_duration, self.congestion['ψ1'], self.congestion['ψ2'])
        
        if self.time_elapsed == 10000: #a scenario lasts for 10,000 steps
            terminal = True
        return state,reward,terminal
    
    def switch_phase(self):
        #switch to the next phase of the cycle Ψ 
        self.current_phase_idx = (self.current_phase_idx + 1) % len(self.signal_cycle) 
        self.current_phase_duration = self.phase_duration[self.signal_cycle[self.current_phase_idx]]
    
    def generate_trafic(self,λ,tau):
        # Generate traffic demand using two Poisson processes
        poisson_1 = np.random.poisson(λ, 4)
        poisson_2 = np.random.poisson(tau, 2)
        self.approaches['N'] += poisson_1[0] + poisson_2[0]
        self.approaches['S'] += poisson_1[1] + poisson_2[1]
        self.approaches['W'] += poisson_1[2]
        self.approaches['E'] += poisson_1[3]
        
    def update_trafic(self):
        depart = 1
        #phase ψ1
        if self.current_phase_idx == 0:
            self.approaches['N'] = max(0,self.approaches['N']- depart)
            self.approaches['S'] = max(0,self.approaches['S']- depart)
        #phase ψ2
        elif self.current_phase_idx == 2:
            self.approaches['W'] = max(0,self.approaches['W']- depart)
            self.approaches['E'] = max(0,self.approaches['E']- depart)
                  
    def update_congestion(self):
        self.congestion['ψ1'] = (self.approaches['N'] + self.approaches['S']) %3
        self.congestion['ψ2'] = (self.approaches['W'] + self.approaches['E']) %3
        return self.congestion

In [38]:
class Agent:
    def __init__(self, tauxApprentissage, gamma, epsilon):
        self.tauxApprentissage = tauxApprentissage
        self.action_space = [0, 1]  # 0 - phase extension of n steps, 1 - switch to next phase
        self.gamma = gamma
        self.epsilon = epsilon
        self.tableauQ = {}

    def choisirAction(self, etat):
        if np.random.uniform() < self.epsilon:
            # Exploration
            return np.random.choice(self.action_space)
        else:
            # Exploitation
            if etat in self.tableauQ:
                return np.argmax(self.tableauQ[etat])
            else:
                return np.random.choice(self.action_space)

    def majTableauQ(self, etat, action, recompense, prochainEtat):
        if etat not in self.tableauQ:
            self.tableauQ[etat] = np.zeros(42)

        if prochainEtat not in self.tableauQ:
            self.tableauQ[prochainEtat] = np.zeros(42)

        ancienneValeur = self.tableauQ[etat][action]
        prochainMax = np.max(self.tableauQ[prochainEtat])
        nouvelleValeur = (1 - self.tauxApprentissage) * ancienneValeur + self.tauxApprentissage * (recompense + self.gamma * prochainMax)
        self.tableauQ[etat][action] = nouvelleValeur


In [39]:
d_min = 5
d_max = 45
n_step = 5

#environnement
intersection = IntersectionEnvironment(d_min,d_max,n_step,controller_type = 'step-based' )

#agent
agent = Agent(tauxApprentissage = 0.05, gamma = 0.05, epsilon = 0.05)

nbiter = 1
for ne in range(nbiter):
    tempsAttenteTotalN = 0
    tempsAttenteTotalS = 0
    tempsAttenteTotalE = 0
    tempsAttenteTotalW = 0
    state = intersection.reset()
    terminal = False
    while not terminal:
        #print("nb de voiture sur l'approche Nord",intersection.approaches["N"])
        action = agent.choisirAction(state)
        next_state,reward, terminal = intersection.step(action)
        
        tempsAttenteTotalN += intersection.approaches["N"]
        tempsAttenteTotalS += intersection.approaches["S"]
        tempsAttenteTotalE += intersection.approaches["E"]
        tempsAttenteTotalW += intersection.approaches["W"]
        
        if next_state[0] != 1 or next_state[0] != 3: #on vérifie que l'on est pas dans un état de transition
            agent.majTableauQ(state,action,reward,state)
        state = next_state
        
    moyenneTempsAttenteE = (tempsAttenteTotalE ) / (10000)
    
    print("/// RESULTATS ///")
    print("Temps d'attente moyen au feu EST :", moyenneTempsAttenteE)

        

 t =  : 0  phase ψ1
 t =  : 1  phase ψ1
 t =  : 2  phase ψ1
 t =  : 3  phase ψ1
 t =  : 4  phase ψ1
 t =  : 5  phase ψ1
 t =  : 6  phase ψ1
 t =  : 7  phase ψ1
 t =  : 8  phase ψ1
 t =  : 9  phase ψ1
 t =  : 10  phase ψ1
 t =  : 11  phase ψ1
 t =  : 12  phase ψ1
 t =  : 13  phase ψ1
 t =  : 14  phase ψ1
 t =  : 15  phase ψ1
 t =  : 16  phase ψ1
 t =  : 17  phase ψ1
 t =  : 18  phase ψ1
 t =  : 19  phase ψ1
 t =  : 20  phase ψ1
phase extended by  5
 t =  : 21  phase ψ1
 t =  : 22  phase ψ1
 t =  : 23  phase ψ1
 t =  : 24  phase ψ1
 t =  : 25  phase ψ1
phase extended by  5
 t =  : 26  phase ψ1
 t =  : 27  phase ψ1
 t =  : 28  phase ψ1
 t =  : 29  phase ψ1
 t =  : 30  phase ψ1
phase extended by  5
 t =  : 31  phase ψ1
 t =  : 32  phase ψ1
 t =  : 33  phase ψ1
 t =  : 34  phase ψ1
 t =  : 35  phase ψ1
phase extended by  5
 t =  : 36  phase ψ1
 t =  : 37  phase ψ1
 t =  : 38  phase ψ1
 t =  : 39  phase ψ1
 t =  : 40  phase ψ1
 t =  : 41  phase ψT
 t =  : 42  phase ψT
 t =  : 43  phase ψT
 t

 t =  : 388  phase ψ2
 t =  : 389  phase ψ2
 t =  : 390  phase ψ2
 t =  : 391  phase ψ2
 t =  : 392  phase ψ2
 t =  : 393  phase ψ2
 t =  : 394  phase ψ2
phase extended by  5
 t =  : 395  phase ψ2
 t =  : 396  phase ψ2
 t =  : 397  phase ψ2
 t =  : 398  phase ψ2
 t =  : 399  phase ψ2
phase extended by  5
 t =  : 400  phase ψ2
 t =  : 401  phase ψ2
 t =  : 402  phase ψ2
 t =  : 403  phase ψ2
 t =  : 404  phase ψ2
phase extended by  5
 t =  : 405  phase ψ2
 t =  : 406  phase ψ2
 t =  : 407  phase ψ2
 t =  : 408  phase ψ2
 t =  : 409  phase ψ2
phase extended by  5
 t =  : 410  phase ψ2
 t =  : 411  phase ψ2
 t =  : 412  phase ψ2
 t =  : 413  phase ψ2
 t =  : 414  phase ψ2
phase extended by  5
 t =  : 415  phase ψ2
 t =  : 416  phase ψ2
 t =  : 417  phase ψ2
 t =  : 418  phase ψ2
 t =  : 419  phase ψ2
 t =  : 420  phase ψT
 t =  : 421  phase ψT
 t =  : 422  phase ψT
 t =  : 423  phase ψT
 t =  : 424  phase ψT
 t =  : 425  phase ψ1
 t =  : 426  phase ψ1
 t =  : 427  phase ψ1
 t =  : 428  ph

 t =  : 780  phase ψ2
 t =  : 781  phase ψ2
 t =  : 782  phase ψ2
 t =  : 783  phase ψ2
 t =  : 784  phase ψ2
phase extended by  5
 t =  : 785  phase ψ2
 t =  : 786  phase ψ2
 t =  : 787  phase ψ2
 t =  : 788  phase ψ2
 t =  : 789  phase ψ2
phase extended by  5
 t =  : 790  phase ψ2
 t =  : 791  phase ψ2
 t =  : 792  phase ψ2
 t =  : 793  phase ψ2
 t =  : 794  phase ψ2
phase extended by  5
 t =  : 795  phase ψ2
 t =  : 796  phase ψ2
 t =  : 797  phase ψ2
 t =  : 798  phase ψ2
 t =  : 799  phase ψ2
phase extended by  5
 t =  : 800  phase ψ2
 t =  : 801  phase ψ2
 t =  : 802  phase ψ2
 t =  : 803  phase ψ2
 t =  : 804  phase ψ2
phase extended by  5
 t =  : 805  phase ψ2
 t =  : 806  phase ψ2
 t =  : 807  phase ψ2
 t =  : 808  phase ψ2
 t =  : 809  phase ψ2
 t =  : 810  phase ψT
 t =  : 811  phase ψT
 t =  : 812  phase ψT
 t =  : 813  phase ψT
 t =  : 814  phase ψT
 t =  : 815  phase ψ1
 t =  : 816  phase ψ1
 t =  : 817  phase ψ1
 t =  : 818  phase ψ1
 t =  : 819  phase ψ1
 t =  : 820  ph

 t =  : 1307  phase ψ2
 t =  : 1308  phase ψ2
 t =  : 1309  phase ψT
 t =  : 1310  phase ψT
 t =  : 1311  phase ψT
 t =  : 1312  phase ψT
 t =  : 1313  phase ψT
 t =  : 1314  phase ψ1
 t =  : 1315  phase ψ1
 t =  : 1316  phase ψ1
 t =  : 1317  phase ψ1
 t =  : 1318  phase ψ1
 t =  : 1319  phase ψ1
 t =  : 1320  phase ψ1
 t =  : 1321  phase ψ1
 t =  : 1322  phase ψ1
 t =  : 1323  phase ψ1
 t =  : 1324  phase ψ1
 t =  : 1325  phase ψ1
 t =  : 1326  phase ψ1
 t =  : 1327  phase ψ1
 t =  : 1328  phase ψ1
 t =  : 1329  phase ψ1
 t =  : 1330  phase ψ1
 t =  : 1331  phase ψ1
 t =  : 1332  phase ψ1
 t =  : 1333  phase ψ1
 t =  : 1334  phase ψ1
phase extended by  5
 t =  : 1335  phase ψ1
 t =  : 1336  phase ψ1
 t =  : 1337  phase ψ1
 t =  : 1338  phase ψ1
 t =  : 1339  phase ψ1
phase extended by  5
 t =  : 1340  phase ψ1
 t =  : 1341  phase ψ1
 t =  : 1342  phase ψ1
 t =  : 1343  phase ψ1
 t =  : 1344  phase ψ1
phase extended by  5
 t =  : 1345  phase ψ1
 t =  : 1346  phase ψ1
 t =  : 1347  pha

 t =  : 1683  phase ψ1
 t =  : 1684  phase ψ1
 t =  : 1685  phase ψ1
 t =  : 1686  phase ψ1
 t =  : 1687  phase ψ1
 t =  : 1688  phase ψ1
 t =  : 1689  phase ψ1
 t =  : 1690  phase ψ1
phase extended by  5
 t =  : 1691  phase ψ1
 t =  : 1692  phase ψ1
 t =  : 1693  phase ψ1
 t =  : 1694  phase ψ1
 t =  : 1695  phase ψ1
phase extended by  5
 t =  : 1696  phase ψ1
 t =  : 1697  phase ψ1
 t =  : 1698  phase ψ1
 t =  : 1699  phase ψ1
 t =  : 1700  phase ψ1
phase extended by  5
 t =  : 1701  phase ψ1
 t =  : 1702  phase ψ1
 t =  : 1703  phase ψ1
 t =  : 1704  phase ψ1
 t =  : 1705  phase ψ1
phase extended by  5
 t =  : 1706  phase ψ1
 t =  : 1707  phase ψ1
 t =  : 1708  phase ψ1
 t =  : 1709  phase ψ1
 t =  : 1710  phase ψ1
 t =  : 1711  phase ψT
 t =  : 1712  phase ψT
 t =  : 1713  phase ψT
 t =  : 1714  phase ψT
 t =  : 1715  phase ψT
 t =  : 1716  phase ψ2
 t =  : 1717  phase ψ2
 t =  : 1718  phase ψ2
 t =  : 1719  phase ψ2
 t =  : 1720  phase ψ2
 t =  : 1721  phase ψ2
 t =  : 1722  phase

...

 t =  : 8404  phase ψ2
 t =  : 8405  phase ψ2
 t =  : 8406  phase ψ2
 t =  : 8407  phase ψ2
 t =  : 8408  phase ψ2
 t =  : 8409  phase ψT
 t =  : 8410  phase ψT
 t =  : 8411  phase ψT
 t =  : 8412  phase ψT
 t =  : 8413  phase ψT
 t =  : 8414  phase ψ1
 t =  : 8415  phase ψ1
 t =  : 8416  phase ψ1
 t =  : 8417  phase ψ1
 t =  : 8418  phase ψ1
 t =  : 8419  phase ψ1
 t =  : 8420  phase ψ1
 t =  : 8421  phase ψ1
 t =  : 8422  phase ψ1
 t =  : 8423  phase ψ1
 t =  : 8424  phase ψ1
 t =  : 8425  phase ψ1
 t =  : 8426  phase ψ1
 t =  : 8427  phase ψ1
 t =  : 8428  phase ψ1
 t =  : 8429  phase ψ1
 t =  : 8430  phase ψ1
 t =  : 8431  phase ψ1
 t =  : 8432  phase ψ1
 t =  : 8433  phase ψ1
 t =  : 8434  phase ψ1
phase extended by  5
 t =  : 8435  phase ψ1
 t =  : 8436  phase ψ1
 t =  : 8437  phase ψ1
 t =  : 8438  phase ψ1
 t =  : 8439  phase ψ1
phase extended by  5
 t =  : 8440  phase ψ1
 t =  : 8441  phase ψ1
 t =  : 8442  phase ψ1
 t =  : 8443  phase ψ1
 t =  : 8444  phase ψ1
phase extended 

 t =  : 8796  phase ψ1
 t =  : 8797  phase ψ1
 t =  : 8798  phase ψ1
 t =  : 8799  phase ψ1
 t =  : 8800  phase ψ1
 t =  : 8801  phase ψ1
 t =  : 8802  phase ψ1
 t =  : 8803  phase ψ1
 t =  : 8804  phase ψ1
 t =  : 8805  phase ψ1
phase extended by  5
 t =  : 8806  phase ψ1
 t =  : 8807  phase ψ1
 t =  : 8808  phase ψ1
 t =  : 8809  phase ψ1
 t =  : 8810  phase ψ1
phase extended by  5
 t =  : 8811  phase ψ1
 t =  : 8812  phase ψ1
 t =  : 8813  phase ψ1
 t =  : 8814  phase ψ1
 t =  : 8815  phase ψ1
phase extended by  5
 t =  : 8816  phase ψ1
 t =  : 8817  phase ψ1
 t =  : 8818  phase ψ1
 t =  : 8819  phase ψ1
 t =  : 8820  phase ψ1
phase extended by  5
 t =  : 8821  phase ψ1
 t =  : 8822  phase ψ1
 t =  : 8823  phase ψ1
 t =  : 8824  phase ψ1
 t =  : 8825  phase ψ1
 t =  : 8826  phase ψT
 t =  : 8827  phase ψT
 t =  : 8828  phase ψT
 t =  : 8829  phase ψT
 t =  : 8830  phase ψT
 t =  : 8831  phase ψ2
 t =  : 8832  phase ψ2
 t =  : 8833  phase ψ2
 t =  : 8834  phase ψ2
 t =  : 8835  phase

 t =  : 9232  phase ψT
 t =  : 9233  phase ψT
 t =  : 9234  phase ψT
 t =  : 9235  phase ψ1
 t =  : 9236  phase ψ1
 t =  : 9237  phase ψ1
 t =  : 9238  phase ψ1
 t =  : 9239  phase ψ1
 t =  : 9240  phase ψ1
 t =  : 9241  phase ψ1
 t =  : 9242  phase ψ1
 t =  : 9243  phase ψ1
 t =  : 9244  phase ψ1
 t =  : 9245  phase ψ1
 t =  : 9246  phase ψ1
 t =  : 9247  phase ψ1
 t =  : 9248  phase ψ1
 t =  : 9249  phase ψ1
 t =  : 9250  phase ψ1
 t =  : 9251  phase ψ1
 t =  : 9252  phase ψ1
 t =  : 9253  phase ψ1
 t =  : 9254  phase ψ1
 t =  : 9255  phase ψ1
phase extended by  5
 t =  : 9256  phase ψ1
 t =  : 9257  phase ψ1
 t =  : 9258  phase ψ1
 t =  : 9259  phase ψ1
 t =  : 9260  phase ψ1
phase extended by  5
 t =  : 9261  phase ψ1
 t =  : 9262  phase ψ1
 t =  : 9263  phase ψ1
 t =  : 9264  phase ψ1
 t =  : 9265  phase ψ1
phase extended by  5
 t =  : 9266  phase ψ1
 t =  : 9267  phase ψ1
 t =  : 9268  phase ψ1
 t =  : 9269  phase ψ1
 t =  : 9270  phase ψ1
phase extended by  5
 t =  : 9271  phase

 t =  : 9606  phase ψ1
 t =  : 9607  phase ψ1
 t =  : 9608  phase ψ1
 t =  : 9609  phase ψ1
 t =  : 9610  phase ψ1
 t =  : 9611  phase ψ1
phase extended by  5
 t =  : 9612  phase ψ1
 t =  : 9613  phase ψ1
 t =  : 9614  phase ψ1
 t =  : 9615  phase ψ1
 t =  : 9616  phase ψ1
phase extended by  5
 t =  : 9617  phase ψ1
 t =  : 9618  phase ψ1
 t =  : 9619  phase ψ1
 t =  : 9620  phase ψ1
 t =  : 9621  phase ψ1
phase extended by  5
 t =  : 9622  phase ψ1
 t =  : 9623  phase ψ1
 t =  : 9624  phase ψ1
 t =  : 9625  phase ψ1
 t =  : 9626  phase ψ1
phase extended by  5
 t =  : 9627  phase ψ1
 t =  : 9628  phase ψ1
 t =  : 9629  phase ψ1
 t =  : 9630  phase ψ1
 t =  : 9631  phase ψ1
 t =  : 9632  phase ψT
 t =  : 9633  phase ψT
 t =  : 9634  phase ψT
 t =  : 9635  phase ψT
 t =  : 9636  phase ψT
 t =  : 9637  phase ψ2
 t =  : 9638  phase ψ2
 t =  : 9639  phase ψ2
 t =  : 9640  phase ψ2
 t =  : 9641  phase ψ2
 t =  : 9642  phase ψ2
 t =  : 9643  phase ψ2
 t =  : 9644  phase ψ2
 t =  : 9645  phase

 t =  : 9997  phase ψ2
 t =  : 9998  phase ψ2
 t =  : 9999  phase ψ2
/// RESULTATS ///
Temps d'attente moyen au feu EST : 3.3724
